## Importing BindingDB

In [ ]:
#This cell imports bindingdb data

from data_interfaces.access_bindingdb import bindingdb_records
import os
from download_file import download
from zipfile import ZipFile

bdb_download_url= "https://www.bindingdb.org/bind/downloads/BindingDB_All_2021m11.tsv.zip"
#if the affinities are already stored from previous run, it just reads the cached (pickled) data
bdb = bindingdb_records()
if (os.path.exists("./data_interfaces/cached_data/bindingdb2021m11_bin_Ki_1uM_30uM.pkl")):
    bdb.read_from_pickle("./data_interfaces/cached_data/bindingdb2021m11_bin_Ki_1uM_30uM.pkl")
else:
    if os.path.exists("data_sources/bindingdb/db2021m11.tsv"):
        pass
    else:
        if not os.path.exists("data_sources/bindingdb/"):
            os.makedirs("data_sources/bindingdb/")
        print("downloading bindingdb file...")
        download(bdb_download_url, "./data_sources/")
        print("extracting bindingdb file...")
        zfilename = os.path.split(bdb_download_url)[1]
        bdb_version = zfilename.split("_")[-1].split(".")[0]
        with ZipFile("./data_sources/"+zfilename,"r") as zip_ref:
            zip_ref.extractall("./data_sources/")
        os.rename("./data_sources/BindingDB_All.tsv","./data_sources/bindingdb/db2021m11.tsv")        
    bdb.read_from_csv("data_sources/bindingdb/db2021m11.tsv")
    bdb.binarize_db(l_threshold=1000,h_threshold=30000)
    if not os.path.exists("./data_interfaces/cached_data/"):
        os.makedirs("./data_interfaces/cached_data/")
    bdb.save_to_pickle("./data_interfaces/cached_data/bindingdb2021m11_bin_Ki_1uM_30uM.pkl")


## Building testset1 (difficult testset)

Here we find cases where a domain interacts with a compound (when it is on a single domain protein), But same domain doesn't interact with that compound when the domain is on a protein with other domains or other unannoated regions. 

we first collect all such cases that could be collected using data from BindingDB and pfam-A

Secondly, we merge the proteins that are similar in their combination/arrangement of domains to make sure we are dealing with different proteins and not proteins that are just a slightly modified version of each other.

testset1 then is a collection of these non-redundant protein-compound pairs that are involved in these 'context-sensitive' interactions. context-sensitive in the sense that the interaction doesn't depend only on the domain of interest but also on the rest of the protein.


In [4]:
import numpy
from data_interfaces.access_pfam import pfam_records
# import importlib
# import fetch_data
# importlib.reload(fetch_data.access_pfam)
from tqdm.autonotebook import tqdm




# then we find all proteins that have that domain and they are divided into pos, neg and unk based on if they interact with that drug or not
# we only have a case when there is an element in neg 
# in that case the domain-drug pair should be added to the pairs and pos_cases[(domain, drug)] is the set of all proteins that have that domain and do interact and neg_cases[(domain, drug)] is the set of proteins with that domain that don't interact. unk_cases will be the set of those that have that domain buut their interaction is not known. But in all of these three dicts only we have keys for the conflicting cases meaning those that have a single domain positive interaction and at least one protein with negative interaction

# first we find all single-domain proteins
pfam_all = pfam_records(prot_list=bdb.get_all_prots(), use_cache_file="data_interfaces/cached_data/pfam_cache.pkl")
single_dom_prots = pfam_all.get_singledom_prots()

pos_dom_cmp = set() # interacting domain-compound pairs
dci_doms = set()
dci_cmps = set()

# if there is a positive interaction between a single-domain protein and a compound, we assume an interaction between domain and compound. We find all interacting domain-compound pairs with this approach
for p in single_dom_prots:
    dom = pfam_all.get_domainlist_for_prot(p)[0]
    ligs = bdb.get_ligands_of(p)
    for lig in ligs:
        pos_dom_cmp.add((dom,lig))
        dci_doms.add(dom)
        dci_cmps.add(lig)
        

        
pos_cases = dict() # positive examples (proteins) for domain-compound pair
neg_cases = dict() # negative examples (proteins) for domain-compound pair
unk_cases = dict() # examples (proteins) with unknown interaction for a domain-compound pair

dff_dom_cmp = set() # domain-compoud pairs that have context-senstive interactions
dff_doms = set() # domains involved in context-sensitive domain-compound pairs
dff_cmps = set() # compounds invloved in context-sensitive domain-compound pairs

dff_pos_pairs = set() # difficult protein-compound pairs - positive
dff_neg_pairs = set() # difficult protein-compound pairs - negative            
                
for (dom,cmp) in pos_dom_cmp:
    q_set = pfam_all.get_proteins_with(dom)
    negatives = []
    positives = []
    unknowns = []
    for q in q_set:
        inn = bdb.get_bin_interaction((q,cmp))
        if type(inn)==bool:
            if inn:
                positives.append(q)
            else:
                negatives.append(q)
        else:
            unknowns.append(q)

    pos_cases[(dom, cmp)] = positives
    neg_cases[(dom, cmp)] = negatives 
    unk_cases[(dom, cmp)] = unknowns
    if len(negatives)>0:
        dff_dom_cmp.add((dom, cmp))
        dff_doms.add(dom)
        dff_cmps.add(cmp)
        for q in negatives:
            dff_neg_pairs.add((q,cmp))
        for q in positives:
            dff_pos_pairs.add((q,cmp))


            
print("results before merging 'similar' proteins ")
print("------------------------------------------")
print("number domain-compound pairs with positive interactions: ", len(pos_dom_cmp))
print("number of domains involved: ", len(dci_doms))
print("number of compounds involved: ", len(dci_cmps))
print("number domain-compound pairs with context-sensitive interactions: ", len(dff_dom_cmp))
print("number of domains involved: ", len (dff_doms))
print("number of comnpounds involved: ", len (dff_cmps))
print("------------------------------------------\n\n")


### here we remove conflicting cases: where 'similar' proteins have found one with negative and one with positive interaction with a drug
### and remove redundancy within positive and negatives 
### to do this:
### for each domain-compound pair:
# we see if there is any intersection between the positive cases and negative cases using our segment string method
# if there is not any, then we reduce the positve set and negative set separately and then when we have our positives and negatives that we then add to the testset1 postive and negative pairs


dff_nred_pos_pairs = set() # non-redundant positive portein-compound pairs
dff_nred_neg_pairs = set() # non-redundant negative portein-compound pairs
dc_pairs_wconflict = set()


for (dom,cmp) in dff_dom_cmp:
    pos_pfset = pfam_all.extract_subset(pos_cases[(dom,cmp)])
#     print(pos_pfset)
    neg_pfset = pfam_all.extract_subset(neg_cases[(dom,cmp)])
    # find conflicting cases where 'similar' proteins are in the positives and negatives
    shared_neg_pos  = pos_pfset.same_prots(neg_pfset)
    if len(shared_neg_pos) > 0:
        dc_pairs_wconflict.add((dom,cmp))
    else:
        #remove redundancy within positive examples and within negative examples
        pref = dict()
        for p in pos_cases[(dom,cmp)]:
            ### when removing redundant proteins, we prefer those with which there is higher number of records for them with the current compound
            pref[p] = len(bdb.get_records_for_pair((p,cmp)))
        pos_nred = pos_pfset.reduce_to_nonredunant(prot_preference_score=pref)
        
        pref = dict()
        for p in neg_cases[(dom,cmp)]:
            ### when removing redundant proteins, we prefer those with which there is higher number of records for them with the current compound
            pref[p] = len(bdb.get_records_for_pair((p,cmp))) 
        neg_nred = neg_pfset.reduce_to_nonredunant(prot_preference_score=pref)
        
        for p in pos_nred:
            dff_nred_pos_pairs.add((p,cmp))
        for p in neg_nred:
            dff_nred_neg_pairs.add((p,cmp))


dff_nred_dc_pairs = dff_dom_cmp - dc_pairs_wconflict # difficult dc pairs remaining after removing conflicting (those that have similar proteins in positive and negative side)



dff_nred_doms = set() # domains involved in the remaining domain-compound pairs after removal of conflicts
dff_nred_cmps = set() # compounds involved in the remaining domain-compound pairs after removal of conflicts
for dom, cmp in dff_nred_dc_pairs:
    dff_nred_doms.add(dom)
    dff_nred_cmps.add(cmp)

    
    
# the set of difficult and non-redundant postive and negative protein-compound pairs will build testset1
dff_nred_pairs = dff_nred_pos_pairs|dff_nred_neg_pairs # protein-compound pairs in testset1

dff_nred_prots =set() #set of proteins involved in testset1. we already have the set of compounds involved from domain-compound pairs
for prot, cmp in dff_nred_pairs:
    dff_nred_prots.add(prot)

print ("\nAfter merging similar proteins")
print("------------------------------------------")
print ("number of domain-ligand pairs: ",len(dff_nred_dc_pairs))
print ("number of number of domains involved: ",len(dff_nred_doms))
print ("number of number of compounds involved: ",len(dff_nred_cmps))
print ("number of resulting protin-compound pairs: ", len(dff_nred_pairs))
print ("number of positive pairs: ", len(dff_nred_pos_pairs))
print ("number of negative pairs: ", len(dff_nred_neg_pairs))
print ("number of protins: ", len(dff_nred_prots))
print("------------------------------------------\n")

testset1_pos_pairs = dff_nred_pos_pairs
testset1_neg_pairs  = dff_nred_neg_pairs
testset1_pairs = dff_nred_pairs
testset1_prots = dff_nred_prots
testset1_cmps = dff_nred_cmps

from build_dataset import save_dataset_csv
if not os.path.exists("./datasets/"):
    os.makedirs("./datasets/")
save_dataset_csv(testset1_pairs,bdb,"datasets/testset1.csv")


collecting domain annotations:   0%|          | 0/5750 [00:00<?, ?it/s]

results before merging 'similar' proteins 
------------------------------------------
number domain-compound pairs with positive interactions:  18229
number of domains involved:  89
number of compounds involved:  18088
number domain-compound pairs with context-sensitive interactions:  660
number of domains involved:  17
number of comnpounds involved:  660
------------------------------------------



After merging similar proteins
------------------------------------------
number of domain-ligand pairs:  256
number of number of domains involved:  9
number of number of compounds involved:  256
number of resulting protin-compound pairs:  739
number of positive pairs:  346
number of negative pairs:  393
number of protins:  67
------------------------------------------



## Building testset2 (proteins and compounds from testset1)

testset2 is created with the assumption that there is something inherently difficult in the proteins and compounds in the testset1. we want the testset2 to  be a superset of testset1 and have similar proportion of positives and negatives to testset1, but also represent the whole interactome of proteins and compounds in testset1

Alternatively we could choose to:
1. to include testset1 in testset2 or exclude it:For now we choose to include it
2. to sample randomly for fixing proportions or to have some bias in their selection. For example towards those proteins and drugs that are involved in more pairs than others: For now we just randomly sample

In [ ]:
protx_prot_cmp_pairs = set()# negative protein-compound pairs that the protein is in testset1
cmpx_prot_cmp_pairs = set() # negative protein-compound pairs where the compound is in the compound set of testset1

ext_prots = set()
ext_cmps = set()



for (prot,cmp) in bdb.get_all_bin_pairs():
    if prot in testset1_prots:
        protx_prot_cmp_pairs.add((prot,cmp))
    if cmp in testset1_cmps:
        cmpx_prot_cmp_pairs.add((prot,cmp))



ext_prot_cmp_pairs = protx_prot_cmp_pairs | cmpx_prot_cmp_pairs

ext_prots = set()
ext_cmps = set()
ext_pos_pairs = set()
ext_neg_pairs = set()

for (prot, cmp) in ext_prot_cmp_pairs:
    ext_cmps.add(cmp)
    ext_prots.add(prot)
    if bdb.get_bin_interaction((prot,cmp)):
        ext_pos_pairs.add((prot,cmp))
    else:
        ext_neg_pairs.add((prot,cmp))
        
        
print ("Extended pairs before resampling")
print ("number of resulting protin-compound pairs: ", len(ext_prot_cmp_pairs))
print ("number of pair from extending by proteins: ", len(protx_prot_cmp_pairs))
print ("number of pair from extending by compounds: ", len(cmpx_prot_cmp_pairs))
print ("number of positive pairs: ", len(ext_pos_pairs))
print ("number of negative pairs: ", len(ext_neg_pairs))
print ("number of protins: ", len(ext_prots))
print ("number of compounds: ", len(ext_cmps))


# we want the testset2 to be a superset of testset1 but also have same postive-negative ratio and 

pos_neg_ratio = len(testset1_pos_pairs)/len(testset1_neg_pairs)

target_neg_number  = int(len(ext_neg_pairs) * 0.5)
target_pos_number = int(target_neg_number * pos_neg_ratio)

n_pos_2samp = target_pos_number-len(testset1_pos_pairs)
n_neg_2samp = target_neg_number-len(testset1_neg_pairs)

pos_samp_pool = ext_pos_pairs-testset1_pos_pairs
neg_samp_pool = ext_neg_pairs-testset1_neg_pairs

import random

sampd_pos = set(random.sample(list(pos_samp_pool), n_pos_2samp))
sampd_neg = set(random.sample(list(neg_samp_pool), n_neg_2samp))

testset2_pos_pairs = testset1_pos_pairs | sampd_pos
testset2_neg_pairs = testset1_neg_pairs | sampd_neg


testset2_pairs = testset2_pos_pairs | testset2_neg_pairs

testset2_prots = set()
testset2_cmps = set()

for (prot,cmp) in testset2_pairs:
    testset2_prots.add(prot)
    testset2_cmps.add(cmp)

print ("\nExtended pairs after resampling (testset2)")
print ("number of protein-compound pairs: ", len(testset2_pairs))
print ("number of positive pairs: ", len(testset2_pos_pairs))
print ("number of negative pairs: ", len(testset2_neg_pairs))
print ("number of proteins: ", len(testset2_prots))
print ("number of compounds: ", len(testset2_cmps))

from build_dataset import save_dataset_csv
if not os.path.exists("./datasets/"):
    os.makedirs("./datasets/")
save_dataset_csv(testset2_pairs,bdb,"datasets/testset2.csv")


# Building testset3
testset3 is intended to be a random sample of all pairs in bindingdb while keeping the ratio between positives and negatives to be the same as their ratio in testset1.
This dataset also have some importance because it is supposed to be a standard and unbiased way of evaluating predictors 
we have some choices here to make:
1- how big the sample should be so it is both representative of a typical outcome of a prediction, not being accused of being based on luck, while at the same time leaving an adequate number of pairs for training (negative pairs are limiting here)


In [ ]:
all_pairs = bdb.get_all_bin_pairs()
all_pos_pairs = set()
all_neg_pairs = set()
all_prots = set()
all_compounds = set()

for (prot,cmp) in all_pairs:
    all_prots.add(prot)
    all_compounds.add(cmp)
    if bdb.get_bin_interaction((prot, cmp)):
        all_pos_pairs.add((prot,cmp))
    else:
        all_neg_pairs.add((prot,cmp))
        
print ("\nAll pairs before resampling:")
print ("number of protein-compound pairs: ", len(all_pairs))
print ("number of proteins: ", len(all_prots))
print ("number of compounds: ", len(all_compounds))
print ("number of positive pairs: ", len(all_pos_pairs))
print ("number of negative pairs: ", len(all_neg_pairs))

pos_neg_ratio = len(testset1_pos_pairs)/len(testset1_neg_pairs)
testset3_target_neg_number  = int(len(all_neg_pairs) * 0.2)
testset3_target_pos_number = int(testset3_target_neg_number * pos_neg_ratio)

import random

testset3_pos_pairs = set(random.sample(list(all_pos_pairs), testset3_target_pos_number))
testset3_neg_pairs = set(random.sample(list(all_neg_pairs), testset3_target_neg_number))
testset3_pairs = testset3_pos_pairs | testset3_neg_pairs

testset3_prots= set()
testset3_compounds= set()
for (prot, cmp) in testset3_pairs:
    testset3_prots.add(prot)
    testset3_compounds.add(cmp)

print ("\ntestset3: (20% sample of all negatives)")
print ("number of protein-compound pairs: ", len(testset3_pairs))
print ("number of proteins: ", len(testset3_prots))
print ("number of compounds: ", len(testset3_compounds))
print ("number of positive pairs: ", len(testset3_pos_pairs))
print ("number of negative pairs: ", len(testset3_neg_pairs))


from build_dataset import save_dataset_csv
if not os.path.exists("./datasets/"):
    os.makedirs("./datasets/")
save_dataset_csv(testset3_pairs,bdb,"datasets/testset3.csv")


## Combined testset (for exclusion from training)

In [ ]:
combtest_neg_pairs = (testset1_neg_pairs | testset2_neg_pairs | testset3_neg_pairs)
combtest_pos_pairs = (testset1_pos_pairs | testset2_pos_pairs | testset3_pos_pairs)
combtest_pairs = combtest_pos_pairs | combtest_neg_pairs


combtest_pairs_prots= set()
combtest_pairs_compounds= set()
for (prot, cmp) in combtest_pairs:
    combtest_pairs_prots.add(prot)
    combtest_pairs_compounds.add(cmp)

print ("\nremaining pairs (not in testset1,2 or 3):")
print ("number of protein-compound pairs: ", len(combtest_pairs))
print ("number of proteins: ", len(combtest_pairs_prots))
print ("number of compounds: ", len(combtest_pairs_compounds))
print ("number of positive pairs: ", len(combtest_pos_pairs))
print ("number of negative pairs: ", len(combtest_neg_pairs))


from build_dataset import save_dataset_csv
if not os.path.exists("./datasets/"):
    os.makedirs("./datasets/")
save_dataset_csv(combtest_pairs, bdb,"datasets/combtestset123.csv")

## shared training set (remaining pairs)

In [ ]:
rem_pos_pairs  = all_pos_pairs - combtest_pos_pairs
rem_pos_pairs  = all_pos_pairs - combtest_pos_pairs
rem_pairs = rem_neg_pairs | rem_pos_pairs

rem_pairs_prots = set()
rem_pairs_compounds = set()

for (prot, cmp) in rem_pairs:
    rem_pairs_prots.add(prot)
    rem_pairs_compounds.add(cmp)

print ("\nremaining pairs (not in testset1,2 or 3):")
print ("number of protein-compound pairs: ", len(rem_pairs))
print ("number of proteins: ", len(rem_pairs_prots))
print ("number of compounds: ", len(rem_pairs_compounds))
print ("number of positive pairs: ", len(rem_pos_pairs))
print ("number of negative pairs: ", len(rem_neg_pairs))


from build_dataset import save_dataset_csv

# save_dataset_csv(rem_pairs, bdb,"datasets/trainset_allrest.csv")




pos_neg_ratio = len(testset1_pos_pairs)/len(testset1_neg_pairs)
trainsr_target_pos_number = int(len(rem_neg_pairs) * pos_neg_ratio)

import random
trainsr_neg_pairs = rem_neg_pairs
trainsr_pos_pairs = set(random.sample(list(rem_pos_pairs), trainsr_target_pos_number))

trainsr_pairs = trainsr_pos_pairs | trainsr_neg_pairs

trainsr_prots= set()
trainsr_compounds= set()
for (prot, cmp) in testset3_pairs:
    trainsr_prots.add(prot)
    trainsr_compounds.add(cmp)

print ("\ntraining set with same pos-neg ratio as testsets")
print ("number of protein-compound pairs: ", len(trainsr_pairs))
print ("number of proteins: ", len(trainsr_prots))
print ("number of compounds: ", len(trainsr_compounds))
print ("number of positive pairs: ", len(trainsr_pos_pairs))
print ("number of negative pairs: ", len(trainsr_neg_pairs))



if not os.path.exists("./datasets/"):
    os.makedirs("./datasets/")
save_dataset_csv(trainsr_pairs,bdb,"datasets/trainset_same_ratio.csv")
